クロスデバイス　Valis拡張との併用

In [1]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

イングリウッド(チェルラーブリリオ)	8279  

In [30]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190308


In [31]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 8279
conversion_group_id = 13907

grp = 'cvg'
#grp = 'rtg'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

SPユーザーをPCにクロスデバイスし、これを種にV拡張

In [32]:
%%slack_notify @yusuke_akada

sd = 'sp'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and conversion_group_id = {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [33]:
df.shape[0]

2043

In [34]:
df.head()

,smn_uid,advertiser_id,device,conversion_group_id
0,2198000068090758,8279,SP,13907
1,777022910526817,8279,SP,13907
2,1986000475938663,8279,SP,13907
3,777021149372677,8279,SP,13907
4,777023105667664,8279,SP,13907


In [35]:
%%slack_notify @yusuke_akada
#XD拡張

xd = 'pc'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [36]:
df_fix.shape[0]

2649

トラスティーパートナーズ(プランテルEX)	8486

In [37]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190308


In [38]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 8486
conversion_group_id = 14163

grp = 'cvg'
#grp = 'rtg'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

SPユーザーをPCにクロスデバイスし、これを種にV拡張

In [39]:
%%slack_notify @yusuke_akada

sd = 'sp'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and conversion_group_id = {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [40]:
df.shape[0]

732

In [41]:
df.head()

,smn_uid,advertiser_id,device,conversion_group_id
0,1149000755849456,8486,SP,14163
1,2220000245627639,8486,SP,14163
2,2068000043182544,8486,SP,14163
3,2188000073373992,8486,SP,14163
4,936000169505137,8486,SP,14163


In [42]:
%%slack_notify @yusuke_akada
#XD拡張

xd = 'pc'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [43]:
df_fix.shape[0]

806

ヴァンテージ(BUBUKA)	8317

In [44]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190308


In [45]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 8317
conversion_group_id = 13940

grp = 'cvg'
#grp = 'rtg'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

SPユーザーをPCにクロスデバイスし、これを種にV拡張

In [46]:
%%slack_notify @yusuke_akada

sd = 'sp'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and conversion_group_id = {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [47]:
df.shape[0]

244

In [48]:
df.head()

,smn_uid,advertiser_id,device,conversion_group_id
0,777021368806429,8317,SP,13940
1,777021598151233,8317,SP,13940
2,1669000070927123,8317,SP,13940
3,1407000223453242,8317,SP,13940
4,777022467737586,8317,SP,13940


In [49]:
%%slack_notify @yusuke_akada
#XD拡張

xd = 'pc'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [50]:
df_fix.shape[0]

406

MOG(イクモア)	8547

In [51]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190308


In [52]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 8547
conversion_group_id = 14291

grp = 'cvg'
#grp = 'rtg'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

SPユーザーをPCにクロスデバイスし、これを種にV拡張

In [53]:
%%slack_notify @yusuke_akada

sd = 'sp'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and conversion_group_id = {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [54]:
df.shape[0]

1225

In [55]:
df.head()

,smn_uid,advertiser_id,device,conversion_group_id
0,777024886711952,8547,SP,14291
1,1647000172709081,8547,SP,14291
2,1085000051527316,8547,SP,14291
3,777021802504002,8547,SP,14291
4,2188000088855199,8547,SP,14291


In [56]:
%%slack_notify @yusuke_akada
#XD拡張

xd = 'pc'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [57]:
df_fix.shape[0]

1537

イングリウッド(Breash)	7939

In [72]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190308


In [76]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 7939
conversion_group_id = 13434

grp = 'cvg'
#grp = 'rtg'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

SPユーザーをPCにクロスデバイスし、これを種にV拡張

※数があまりにも少ないので、種はPC/SP問わずに、直近60日に

In [81]:
%%slack_notify @yusuke_akada

sd = 'sp'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-60)) and to_date(days_add(now(),-1))
    and conversion_group_id = {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [82]:
df.shape[0]

73

In [83]:
df.head()

,smn_uid,advertiser_id,device,conversion_group_id
0,2206000054753276,7939,SP,13434
1,1103000005901172,7939,SP,13434
2,2164000016573932,7939,SP,13434
3,1647000110215130,7939,SP,13434
4,692000022416798,7939,SP,13434


In [84]:
%%slack_notify @yusuke_akada
#XD拡張

xd = 'pc'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [85]:
df_fix.shape[0]

199

アドネクスト(KOMBUCHA)	8517

In [65]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190308


In [66]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 8517
conversion_group_id = 14204

grp = 'cvg'
#grp = 'rtg'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

SPユーザーをPCにクロスデバイスし、これを種にV拡張

In [67]:
%%slack_notify @yusuke_akada

sd = 'sp'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and conversion_group_id = {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [68]:
df.shape[0]

9100

In [69]:
df.head()

,smn_uid,advertiser_id,device,conversion_group_id
0,1149000828997702,8517,SP,14204
1,777024774324688,8517,SP,14204
2,674000004916387,8517,SP,14204
3,777023929481788,8517,SP,14204
4,777022927488266,8517,SP,14204


In [70]:
%%slack_notify @yusuke_akada
#XD拡張

xd = 'pc'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [71]:
df_fix.shape[0]

12915

SPユーザーをPCにクロスデバイスし、これを種にV拡張

%%slack_notify @yusuke_akada

sd = 'sp'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and conversion_group_id = {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

df.shape[0]

df.head()

%%slack_notify @yusuke_akada
#XD拡張

xd = 'pc'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device not in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False)

df_fix.shape[0]